# Comparing the Hamiltonians for spin down fixed and free

In [1]:
import numpy as np
import pandas as pd
from qiskit import quantum_info as qi

First we need to generate the fixed states.  This will be used for the spin up sector in both Hamiltonians and for the spin down sector in the fixed Hamiltonian.  It will also help us to convert between the two Hamiltonians.

In [2]:
#Creates all states with N fermions and S orbitals
def Generate_States(N,S):
    s = [i for i in range(0,N)]
    psi_l = []
    while s[0] < S-N:
        ### create the state and store it
        psi = 0
        for i in s:
            psi += 2**i
        psi_l.append(psi)
        ###
        ###Update the particle locations
        exit = 0
        i = len(s)-1
        while exit == 0:
            if s[i] < S-len(s)+i:
                s[i] += 1
                for j in range(i+1,len(s)):
                    s[j] = s[j-1]+1
                exit = 1
            else:
                i -= 1
        ###
    ###Create the finale state
    psi = 0
    for i in s:
        psi += 2**i
    psi_l.append(psi)
    ###
    return psi_l

states = Generate_States(2,4)
states

[3, 5, 9, 6, 10, 12]

The states array holds a list of integeres whose binary representation corresponds to a state with the desired number of spins (two in this example).  We can use the binary representation of these integers to form the spin up sector of both Hamiltonian and the spin down sector of the fixed Hamiltoniain.  The integers themselves tell us which states in the full Hilbert space correspond to states in the fixed Hilbert space.  

Because we are interested in the binary representation of the integers in 'states' let us build a function which returns the binary representation without the prefactos that python likes to include.  

In [3]:
# A function to print out the binary representation of num given the total number of sites S
def bi(num,S):
    bi = bin(num)
    out = ""
    Sdiff = S - len(bi) + 2
    for i in range(0,Sdiff):
        out = out + '0'
    for i in range(2,len(bi)):
        out = out + bi[i]
    return out

bi(states[4],4)

'1010'

# Kinetic Part for a fixed spin

Now let us generate the kinetic part of the Hamiltonian for a fixed number of spins.  This will be used in the spin up part of both Hamiltonians and the spin down part of the fixed Hamiltonian

In [4]:
#Given a state V and the number of sites Su, returns matrix elements of K 
#in the form of [Vk,t] where Vk is the new vector and t is the matrix element t = <Vk|K|V>
def bi_t(V,Su):
    Vn_l = []
    sign = 0
    for i in range(0,Su):
        #Spin up
        M = 2**i + 2**np.mod(i+1,Su)
        K = M & V
        L = K ^ M
        if L != 0 and L != M:
            Vn = V - K + L
            if i + 1 == np.mod(i+1,Su):
                sign = 1
            elif Su % 2 == 1:
                sign = 1
            else:
                sign = -1
            #print(i,':',bi(Vn))
            Vn_l.append([Vn,sign])
    return Vn_l

#print(bi_t(states[4],4))

#Returns the matrix representation of K
t = -1
S = 4
Q = len(states)
index_map = {states[i]:i for i in range(Q)}
K_up = np.array([[0 for i in range(Q)] for j in range(Q)])
for i in range(0,Q):
    psi_t = bi_t(states[i],S)
    for s in range(len(psi_t)):
        K_up[index_map[psi_t[s][0]],i] = psi_t[s][1]*t


        
K_up

array([[ 0, -1,  0,  0,  1,  0],
       [-1,  0, -1, -1,  0,  1],
       [ 0, -1,  0,  0, -1,  0],
       [ 0, -1,  0,  0, -1,  0],
       [ 1,  0, -1, -1,  0, -1],
       [ 0,  1,  0,  0, -1,  0]])

# Free Hamiltonian

## number operator

In [5]:
# returns the matrix element <V|n_i|V> given the total number of sites S
def bi_n(i,V,S):
    num = bi(V,S)
    if num[-i-1] == '1':
        return 1
    else:
        return 0
#print(bi(states[2],4))
#print(bi_n(0,states[2],4))

# returns an array which holds the value of n_{ci}
u = 1
S = 4
Q = len(states)
n_up = []
for c in range(0,Q):
    n_up_c = []
    for i in range(0,S):
        n_up_c.append(u*bi_n(i,states[c],S))
    n_up.append(n_up_c)
    
n_up

[[1, 1, 0, 0],
 [1, 0, 1, 0],
 [1, 0, 0, 1],
 [0, 1, 1, 0],
 [0, 1, 0, 1],
 [0, 0, 1, 1]]

## Blocks

In [6]:
#Identity for the spin-down sector
I_down = [[0 for i in range(2**S)] for j in range(2**S)]
for i in range(2**S):
    I_down[i][i] = 1
I_down = np.array(I_down)
    
#Label for the Z part of the Hamiltonian
def Z_label(i,Q):
    out = ''
    for j in range(0,Q):
        if j == i:
            out = out + 'Z'
        else:
            out = out + 'I'
    return out[::-1]


#Label for the X part of the Hamiltonain
def X_label(i,Q):
    out = ''
    for j in range(0,Q):
        if i < Q-1:
            if i == j or i + 1 == j:
                out = out + 'X'
            else:
                out = out + 'I'
        else:
            if j == 0 or j == Q-1:
                out = out + 'X'
            else:
                out = out + 'Z'
    return out[::-1]

#Label for the Y part of the Hamiltonian
def Y_label(i,Q):
    out = ''
    for j in range(0,Q):
        if i < Q-1:
            if i == j or i + 1 == j:
                out = out + 'Y'
            else:
                out = out + 'I'
        else:
            if j == 0 or j == Q-1:
                out = out + 'Y'
            else:
                out = out + 'Z'
    return out[::-1]
    
#Hamltonian for block c, cc
def Hcc(c,cc):
    h = K_up[c][cc]*I_down
    if c == cc:
        for i in range(0,S): 
            h = h + 1/2*t*qi.Operator.from_label(X_label(i,S)).data
            h = h + 1/2*t*qi.Operator.from_label(Y_label(i,S)).data
            h = h + 1/2*u*n_up[c][i]*(I_down - qi.Operator.from_label(Z_label(i,S)).data)
    return np.array(h)
            


## Full Hamiltonian

In [7]:
Nc = 6
Ndown = 2**S
H_free = [[0 for i in range(Nc*Ndown)] for j in range(Nc*Ndown)]
for c in range(Nc):
    for i in range(Ndown):
        for cc in range(Nc):
            for j in range(Ndown):
                H_free[Ndown*c + i][Ndown*cc + j] = Hcc(c,cc)[i][j]

# Fixed Hamiltonian

## Kinetic part

In [8]:
I0 = np.identity(len(K_up))
HK_fix = np.kron(K_up,I0) + np.kron(I0,K_up)

## Interaction part

In [151]:
#States for both spin sectors together
states_full = []
for i in range(0,len(states)):
    for j in range(0,len(states)):
        states_full.append(states[i]*2**S+states[j])

        
#Returns a list [V,V,..] where the number of times V is in the list corresponds to the
#number of spin pairs.
def bi_u(V,S):
    Su = int(S/2)
    Vn_l = []
    for i in range(0,Su):
        M = 2**i + 2**(i+Su)
        K = M & V
        if K == M:
            Vn = V
            Vn_l.append(Vn)
    return Vn_l
#print(states_full)
#print(bi_u(states_full[3],6))

#the interaction part of the Hamiltonian
u = 0.0
S = 4
Qf = len(states_full)
index_map = {states_full[i]:i for i in range(Qf)}
Hu_fix = np.array([[0 for i in range(Qf)] for j in range(Qf)])
for i in range(0,Qf):
    psi_t = bi_u(states_full[i],2*S)
    for s in range(len(psi_t)):
        Hu_fix[index_map[psi_t[s]],i] += u

## Full Hamiltonian

In [152]:
H_fix = HK_fix + Hu_fix

# Compare the two Hamiltonians

We can compare the matrix elements of the Hamiltonians using the 'states' list.  The matrix element $H^{fixed}_{ci,dj}$ of the fixed Hamiltonian in block $(c,d)$ is mapped to the matrix element $H^{free}_{c~\text{states}[i],d~\text{states}[j]}$ for the free Hamiltoniain in the same block.

In [153]:
c=1
cc=1 
i=1
ii=2
print(bi(states[c],4))
print(bi(states[cc],4))
print(bi(states[i],4))
print(bi(states[ii],4))
print(H_fix[c*6+i,cc*6+ii])
print(Hcc(c,cc)[states[i],states[ii]])
print(H_free[c*2**4 + states[i]][cc*2**4 + states[ii]])

0101
0101
0101
1001
-1.0
(-1+0j)
(-1+0j)


We can also compare the energy levels.  Every energy level of the fixed Hamltoniain should be an energy level of the free Hamiltoniain

In [154]:
e_free,y_free = np.linalg.eig(H_free)

np.sort(e_free)

array([-3.55405387e+00-8.02203439e-24j, -3.55405387e+00-7.88860905e-31j,
       -3.34084762e+00+4.68039320e-17j, -3.29295138e+00-1.30140565e-16j,
       -2.86387634e+00-1.38371217e-17j, -2.78526086e+00+1.38372085e-17j,
       -2.55405387e+00+8.41796450e-18j, -2.55405387e+00-3.90292317e-20j,
       -2.00000000e+00+2.76781780e-17j, -2.00000000e+00+0.00000000e+00j,
       -2.00000000e+00-6.92669627e-23j, -1.79128785e+00-1.62015824e-16j,
       -1.79128785e+00+6.66133815e-16j, -1.56155281e+00-1.08420629e-19j,
       -1.56155281e+00+4.86762111e-18j, -1.30277564e+00+7.81015311e-20j,
       -1.30277564e+00-2.30479106e-17j, -1.00000000e+00-1.00918417e-25j,
       -1.00000000e+00-9.30787565e-17j, -1.00000000e+00-1.83465486e-20j,
       -1.00000000e+00+3.55313405e-28j, -1.00000000e+00+4.17574818e-17j,
       -1.00000000e+00-1.97911182e-18j, -7.91287847e-01+1.12292746e-18j,
       -7.91287847e-01-4.04187442e-18j, -5.61552813e-01+4.37338057e-17j,
       -5.61552813e-01-1.26080837e-17j, -3.02775638

In [155]:
e_fix,y_fix = np.linalg.eig(H_fix)

np.sort(e_fix)

array([-4.00000000e+00+0.00000000e+00j, -4.00000000e+00+0.00000000e+00j,
       -4.00000000e+00+0.00000000e+00j, -4.00000000e+00+0.00000000e+00j,
       -2.00000000e+00+0.00000000e+00j, -2.00000000e+00+0.00000000e+00j,
       -2.00000000e+00+0.00000000e+00j, -2.00000000e+00+0.00000000e+00j,
       -2.00000000e+00+0.00000000e+00j, -2.00000000e+00+0.00000000e+00j,
       -2.00000000e+00+0.00000000e+00j, -2.00000000e+00+0.00000000e+00j,
       -1.79535360e-16+0.00000000e+00j, -1.70216296e-16+0.00000000e+00j,
       -1.61812771e-16+0.00000000e+00j, -3.55773415e-17+0.00000000e+00j,
       -2.18015587e-17-1.29000861e-16j, -2.18015587e-17+1.29000861e-16j,
        3.57259426e-17-2.04654417e-16j,  3.57259426e-17+2.04654417e-16j,
        1.34561858e-16+0.00000000e+00j,  3.17067822e-16+0.00000000e+00j,
        3.40097149e-16+0.00000000e+00j,  6.82375074e-16+0.00000000e+00j,
        2.00000000e+00+0.00000000e+00j,  2.00000000e+00+0.00000000e+00j,
        2.00000000e+00+0.00000000e+00j,  2.00000000

In [156]:
match = []
for ex in e_fix:
    test = 0
    for er in e_free:
        if ex - 10**(-8) < er < ex + 10**(-8):
            test = 1
    if test > 0.5 : 
        match.append('yes')
        
match     

['yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes']

In [173]:
e_fix,y_fix = np.linalg.eig(H_fix)
psi_fix = np.transpose(y_fix)
order = np.argsort(e_fix)

print(e_fix[order[0]])
np.dot(np.conjugate(psi_fix[order[0]]),np.dot(H_fix,psi_fix[order[0]]))

(-4.000000000000005+0j)


(-3.9999999999999996+0j)

In [178]:
psi_fix[order[0]]

array([-0.00124696+0.j,  0.13958669+0.j,  0.14083366+0.j,  0.14083366+0.j,
        0.14208062+0.j,  0.00124696+0.j,  0.18988896+0.j,  0.25241945+0.j,
        0.06253049+0.j,  0.06253049+0.j, -0.12735847+0.j, -0.18988896+0.j,
        0.19113592+0.j,  0.11283276+0.j, -0.07830317+0.j, -0.07830317+0.j,
       -0.26943909+0.j, -0.19113592+0.j,  0.19113592+0.j,  0.11283276+0.j,
       -0.07830317+0.j, -0.07830317+0.j, -0.26943909+0.j, -0.19113592+0.j,
        0.19238289+0.j, -0.02675394+0.j, -0.21913682+0.j, -0.21913682+0.j,
       -0.41151971+0.j, -0.19238289+0.j,  0.00124696+0.j, -0.13958669+0.j,
       -0.14083366+0.j, -0.14083366+0.j, -0.14208062+0.j, -0.00124696+0.j])

In [177]:
bi(states[0],4)

'0011'

In [160]:
N=2
print(1/np.sqrt(N)*np.sin(1*np.pi/N))
print(1/np.sqrt(N)*np.sin(2*np.pi/N))
print(1/np.sqrt(N)*np.sin(3*np.pi/N))
print(1/np.sqrt(N)*np.sin(4*np.pi/N))
print(1/np.sqrt(N)*np.sin(5*np.pi/N))

0.7071067811865475
8.659560562354932e-17
-0.7071067811865475
-1.7319121124709863e-16
0.7071067811865475


In [161]:
np.dot(psi_fix[0],psi_fix[0])

0.9999999999999998

In [167]:
np.sort(e_fix)

array([-4.00000000e+00+0.00000000e+00j, -4.00000000e+00+0.00000000e+00j,
       -4.00000000e+00+0.00000000e+00j, -4.00000000e+00+0.00000000e+00j,
       -2.00000000e+00+0.00000000e+00j, -2.00000000e+00+0.00000000e+00j,
       -2.00000000e+00+0.00000000e+00j, -2.00000000e+00+0.00000000e+00j,
       -2.00000000e+00+0.00000000e+00j, -2.00000000e+00+0.00000000e+00j,
       -2.00000000e+00+0.00000000e+00j, -2.00000000e+00+0.00000000e+00j,
       -1.79535360e-16+0.00000000e+00j, -1.70216296e-16+0.00000000e+00j,
       -1.61812771e-16+0.00000000e+00j, -3.55773415e-17+0.00000000e+00j,
       -2.18015587e-17-1.29000861e-16j, -2.18015587e-17+1.29000861e-16j,
        3.57259426e-17-2.04654417e-16j,  3.57259426e-17+2.04654417e-16j,
        1.34561858e-16+0.00000000e+00j,  3.17067822e-16+0.00000000e+00j,
        3.40097149e-16+0.00000000e+00j,  6.82375074e-16+0.00000000e+00j,
        2.00000000e+00+0.00000000e+00j,  2.00000000e+00+0.00000000e+00j,
        2.00000000e+00+0.00000000e+00j,  2.00000000

In [191]:
def findx(V,S):
    vl = bi(V,S)
    out = []
    for i,n in enumerate(vl):
        #print(n)
        if n == '1':
            out.append(S-1-i)
    return out

findx(states[3],4)

[2, 1]

In [293]:
gutz = []
for V in states:
    x = findx(V,S)
    x1 = x[0]
    x2 = x[1]
    gutz.append(1/2*np.exp(1j*(x1+x2)*np.pi/4)*np.sin((x1-x2)*np.pi/4))
    
print(np.dot(np.conjugate(gutz),gutz))
print(np.amax(np.abs(np.dot(K_up,gutz)+2*np.array(gutz))))

(1+0j)
1.2412670766236366e-16


In [294]:
gutz

[(0.25+0.24999999999999994j),
 (3.061616997868383e-17+0.5j),
 (-0.25+0.25000000000000006j),
 (-0.24999999999999994+0.25j),
 (-0.5+6.123233995736766e-17j),
 (-0.25000000000000006-0.24999999999999994j)]

In [254]:
K_up

array([[ 0, -1,  0,  0,  1,  0],
       [-1,  0, -1, -1,  0,  1],
       [ 0, -1,  0,  0, -1,  0],
       [ 0, -1,  0,  0, -1,  0],
       [ 1,  0, -1, -1,  0, -1],
       [ 0,  1,  0,  0, -1,  0]])

In [257]:
bi(states[2],S)

'1001'

In [193]:
e_k,y_k = np.linalg.eig(K_up)
psi_k = np.transpose(y_k)
order = np.argsort(e_k)

print(e_k[order[0]])
np.dot(np.conjugate(psi_k[order[0]]),np.dot(K_up,psi_k[order[0]]))

-2.0000000000000004


-2.0000000000000004

In [222]:
e_k

array([ 4.4408921e-16,  2.0000000e+00, -2.0000000e+00, -2.0000000e+00,
        2.0000000e+00,  0.0000000e+00])